In [7]:
import pandas as pd
import datetime as dt
import numpy as np

In [8]:
IN_PATH = '/Users/anokhin/Desktop/sample_mail_ok_vk_profi_raw_1120.tsv'
OUT_PATH_LEVELS = '/Users/anokhin/Desktop/profi_levels.la.jss'
OUT_PATH_COSTS = '/Users/anokhin/Desktop/profi_costs.la.jss'

In [9]:
ts_parser = lambda ts: dt.datetime.strptime(ts, "%Y%m%d")
df1 = pd.read_csv(
    IN_PATH, sep="\t", 
    header=None, names=['cuid', 'emails', 'okids', 'vkids', 'retro_dt', 'level', 'cost'],
    converters={'retro_dt': ts_parser},
    dtype={"okids": str, "vkids": str}
)
print df1["retro_dt"].min(), df1["retro_dt"].max()

df1.head()

2015-08-01 00:00:00 2015-12-31 00:00:00


,cuid,emails,okids,vkids,retro_dt,level,cost
0,5003848,NaN,NaN,252551709,2015-11-20,1,-
1,5052798,NaN,NaN,323508874,2015-11-29,3,1
2,4794538,NaN,NaN,160637104,2015-10-17,3,-
3,4704168,NaN,NaN,205539126,2015-10-04,5,4
4,4855914,NaN,391794827,246475336,2015-10-27,3,2


In [17]:
USER_PATTERN = '{"vid":"%s","okid":%s,"email":%s,"vkid":%s,"category":{"int":%s},"start":{"long":%d},"end":{"long":%d}}'
STRING_PATTERN = '{"string":"%s"}'
MONTH_SECONDS = 31 * 24 * 60 * 60

In [18]:
with open(OUT_PATH_LEVELS, "w") as out_file_levels:
    with open(OUT_PATH_COSTS, "w") as out_file_costs:
        for i, row in df1.iterrows():
            end_ts = int(row["retro_dt"].strftime("%s"))
            start_ts = end_ts - MONTH_SECONDS        

            emails = row["emails"].split(",") if pd.notnull(row["emails"]) else [""]
            okids = row["okids"].split(",") if pd.notnull(row["okids"]) else [""]
            vkids = row["vkids"].split(",") if pd.notnull(row["vkids"]) else [""]
            
            level = row["level"]
            cost = row["cost"]

            for email in emails:
                for okid in okids:
                    for vkid in vkids:
                        em = STRING_PATTERN % email if email else "null"
                        ok = STRING_PATTERN % okid if okid else "null"
                        vk = STRING_PATTERN % vkid if vkid else "null"                        
                        
                        if level != "-":
                            print >> out_file_levels, USER_PATTERN % ("", ok, em, vk, int(level), start_ts, end_ts)
                            
                        if cost != "-":
                            print >> out_file_costs, USER_PATTERN % ("", ok, em, vk, int(cost), start_ts, end_ts)

In [19]:
USER_PATTERN = '{"vid":"%s","okid":%s,"email":%s,"vkid":null,"category":{"int":%d},"start":{"long":%d},"end":{"long":%d}}'
with open("/Users/anokhin/Desktop/active_emails.jss", "w") as out_file:
    with open("/Users/anokhin/Desktop/active_emails_1") as ae:
        for line in ae:
            print >> out_file, USER_PATTERN % ("", "null", STRING_PATTERN % line.strip(), 6, 1446163200, 1448841600)